In [1]:
#We need to install tf version 2.15 for compatibility with this notebook (written in March 2024).
!pip uninstall tensorflow
!pip install tensorflow==2.15.0
!tensorflow --version

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.17.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.17.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
  Att

# Transfer Learning with TensorFlow part 3 - Scaling Up (Food Vision Mini)

We've seen the power of transfer learning ,feature extraction, and fine tuning.  Now it's time to scale up to all of the classes in Food101... 101 total classes of food.  Our goal is to beat the original Food101 paper with 10% of the training data, leveraging the power of deep learning.

Original Food101 paper: https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/static/bossard_eccv14_food-101.pdf

Our baseline to beat is 50.76% accuracy across 101 classes.

In [2]:
!nvidia-smi

Fri Nov  8 04:39:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Creating helper functions

In previous notebooks we have created a series of helper functions to help us in the task.  Let's download them.

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-11-08 04:39:34--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-11-08 04:39:34 (115 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
# Import series of helper functions for our notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir



## 101 Food Classes: working with less data

Our goal is to beat the original Food101 paper with 10% of the training data.  Let's download it.

The data we're downloading comes from the original Food101 data set, but has been preprocessed using the image_data_modification notebook.

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
unzip_data("101_food_classes_10_percent.zip")

train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

--2024-11-08 04:39:40--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.207, 64.233.170.207, 142.251.175.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

ent.zip              47%[========>           ] 737.93M  14.8MB/s    eta 54s    

In [ ]:
# How many images classes are there?
walk_through_dir("101_food_classes_10_percent")

In [ ]:
# Setup data inputs
import tensorflow as tf
IMG_SIZE=(224,224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE)

test_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                               label_mode="categorical",
                                                                               image_size=IMG_SIZE,
                                                                               shuffle=False)     #don't shuffle test data for prediction analysis.

## Train a big dog model with transfer learning on 10% of 101 food classes

Here are the steps we're going to take:
* Create a ModelCheckpoint callback
* Create a data augmentation layer to build data augmentation right into the model
* Build a headless (no top layer) Functional EfficientNetB0 backboned-model (we'll create our own output layer)
* Compile our model
* Feature extract for 5 full passes (5 passes on the train dataset and validate on 15% of the test data to save epoch time)

In [ ]:
# Create CheckPoint callback
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor="val_accuracy",
                                                         save_best_only=True)

In [ ]:
# Create data augmentation layer to incorporate it right into our model
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Setup data augmentation
data_augmentation = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2),
    #preprocessing.Rescale(1/255.), #rescale inputs of images to between 0 and 1, required for models like ResNet50
], name="data_augmentation")

In [ ]:
# Setup the base model and freeze its layers (this will extract features)
base_model=tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

#Setup model architecture with trainable top layers
inputs = layers.Input(shape=(224,224,3), name="input_layer")
x = data_augmentation(inputs) # augment images (only happens during training phase)
x = base_model(x, training=False) # This will put the base model into inference mode so weights which need to stay frozen stay frozen
x = layers.GlobalAveragePooling2D(name="global_avg_pool_layer")(x)
outputs = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name="output_layer")(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
# Get a summary of model we've created
model.summary()

In [ ]:
initial_epochs = 5

In [ ]:
# Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics="accuracy",)



history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                    batch_size=32,
                    epochs=initial_epochs,
                    validation_data=test_data_all_10_percent,
                    validation_steps=int(0.15 * len(test_data_all_10_percent)),
                    callbacks=[checkpoint_callback])

In [ ]:
feature_extraction_results = model.evaluate(test_data_all_10_percent)
feature_extraction_results

In [ ]:
plot_loss_curves(history_all_classes_10_percent)

** QUestion: ** What do these two curves suggest?  Hint: Ideally these two curves should be very similar to each other.  IF not, it may suggest our model is overfitting... performing too well on training data and not generalizing to unseen data.


## Fine Tuning

In [ ]:
# Freeze all of the layers in the base model
base_model.trainable = False

# Unfreeze last 5
for layer in base_model.layers[-5:]:
  layer.trainable = True

In [ ]:
# Recompile model with lower learning rate (it's typically best practice the LR when fine-tuning by 10x)
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(lr=0.0001), #learning rate lowered by 10x
              metrics="accuracy")

# What layers in the model are trainable
for layer in model.layers:
  print(layer.name, layer.trainable, )

In [ ]:
# Check which layers are trainable in our base model
for layer_number, layer in enumerate(model.layers[2].layers):
  print (layer_number, layer.name, layer.trainable)

In [ ]:
# Fine tune for another 5 epochs
fine_tuning_epochs = initial_epochs + 5

history_fine_all_data_10_percent = model.fit(train_data_all_10_percent,
                                             batch_size=32,
                                             epochs=fine_tuning_epochs,
                                             validation_data=test_data_all_10_percent,
                                             validation_steps=int(0.15 * len(test_data_all_10_percent)),
                                             initial_epoch=history_all_classes_10_percent.epoch[-1])

In [ ]:
# Evaluate on the whole test data
all_classes_10_percent_fine_tune_results = model.evaluate(test_data_all_10_percent)

In [ ]:
all_classes_10_percent_fine_tune_results

In [ ]:
# Compare histories of feature extraction model with fine tuning model
compare_historys(original_history=history_all_classes_10_percent,
                 new_history=history_fine_all_data_10_percent,
                 initial_epochs=5)

## Saving and loading our model
To use our model in an external application, we will need to save it and export it somewhere


In [ ]:
# Save our fine-tuned model
model.save("drive/MyDrive/TensorFlowCourse/101_food_classes_10_percent_saved_big_dog_model")

In [ ]:
# Load and evaluate saved model
loaded_model = tf.keras.models.load_model("drive/MyDrive/TensorFlowCourse/101_food_classes_10_percent_saved_big_dog_model")

In [ ]:
# Evaluate loaded model and compare performance to pre-saved model
loaded_model_results = loaded_model.evaluate(test_data_all_10_percent)
loaded_model_results

In [ ]:
# The results from our loaded model (above) should be very similar to the results below
all_classes_10_percent_fine_tune_results

## Evaluating the performance of the big dog model across all different classes

Let's make some predictions, visualize them, and then later find out which predictions were the "most" wrong

In [ ]:
import tensorflow as tf
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip



In [ ]:
unzip_data("/content/06_101_food_class_10_percent_saved_big_dog_model.zip")

In [ ]:
# Load in saved model (one that was saved earlier) so that all predictions are similar
loaded_model = tf.keras.models.load_model("/content/06_101_food_class_10_percent_saved_big_dog_model")

In [ ]:
# Evaluate loaded model (the one we just downloaded) on test data
results_downloaded_model = model.evaluate(test_data_all_10_percent)
results_downloaded_model

## Making predictions with our trained model


In [ ]:
# Make predictions with our model
pred_probs = model.predict(test_data_all_10_percent,
                           verbose=1) # Set verbosity to see how long is left

In [ ]:
# How many predictions are there?
len(pred_probs)

In [ ]:
# What's the shape of our predictions?
pred_probs.shape

In [ ]:
# Let's see what the first 10 predictions look like
pred_probs[:10]

In [ ]:
# What does the first prediction probability array look like?
pred_probs[0], len(pred_probs[0]), sum(pred_probs[0])

Our model outputs a prediction probability array with n-number of variables where n is the number of classes of each sample passed to the predict method.

In [ ]:
# We get one probability prediction per class (in our case there's 101 prediction probabilities)
print(f"Number of prediction probabilities for sample 0: {len(pred_probs[0])}")
print(f"What prediction probability sample 0 looks like: \n {pred_probs[0]}")
print(f"The class with the highest predicted probability by the model for sample 0: {pred_probs[0].argmax()}")

In [ ]:
# Get the pred classes of each label
pred_classes = pred_probs.argmax(axis=1)

# How do they look?
pred_classes[:10]

In [ ]:
# How many pred classes do we have?
len(pred_classes)

In [ ]:
test_data_all_10_percent.class_names[pred_classes[9]]

Now we've got a predictions array of all our model's predictions.  To evaluate them we need to compare them to the original test dataset labels.

In [ ]:
# To get our test labels, we need to unravel our test_data BatchDataSet
y_labels = []
test_data_all_10_percent
for images, labels in test_data_all_10_percent.unbatch():
  y_labels.append(labels.numpy().argmax()) # currently test labels look like [0,0,0,1,0,...] we want the index value where 1 occurs
y_labels[:10] # look at the first 10


In [ ]:
len(y_labels)

In [ ]:
len(test_data_all_10_percent)

## Evaluating our model's predicitons
One way to predict if our model's predictions array is in the same order as our test labels array is to find the accuracy score.

In [ ]:
results_downloaded_model

In [ ]:
# Let's try scikit-learn's accuracy score function and see what it comes up with
from sklearn.metrics import accuracy_score
sklearn_accuracy = accuracy_score(y_true=y_labels,
                                  y_pred=pred_classes)
sklearn_accuracy

In [ ]:
# Does this metric come close to our model's evaluate results?
import numpy as np
np.isclose(results_downloaded_model[1], sklearn_accuracy)

# Let's get visual: making a confusion matrix

In [ ]:
# Get a list of class names
class_names = test_data_all_10_percent.class_names
class_names

In [ ]:
len(y_labels)

In [ ]:
# We need to make some changes to our make_confusion_matrix to make our x-labels print vertically
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).

  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes),
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)

  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  ###Changed (plot x-labels vertically):###
  plt.xticks(rotation=70, fontsize=text_size)
  plt.yticks(fontsize=text_size)

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")

In [ ]:
#from helper_functions import make_confusion_matrix
make_confusion_matrix(y_true=y_labels,
                      y_pred=pred_classes,
                      classes=test_data_all_10_percent.class_names,
                      figsize=(50,50),
                      text_size=15,
                      savefig=True)

## Let's keep the evaluation train going - time for a classification report

Scikit-learn has a helpful function for acquiring many different classification metrics per class (eg. precision, recall, and F1) called classification report (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)



In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true=y_labels,
                            y_pred=pred_classes))


The numbers above give a great class-by-class evaluation of our model's predictions but with so many classes, they're quite hard to understand.  

How about we create a visualization to get a better understanding.  

In [ ]:
# Get a dictionary of the classification report
classification_report_dict = classification_report(y_labels, pred_classes, output_dict=True)
classification_report_dict

In [ ]:
class_names[70]

Let's plot all our F1 scores

In [ ]:
# Create an empty dictionary
class_f1_scores = {}
# Loop through classification report dictionary items
for k, v in classification_report_dict.items():
  if k == "accuracy":  #stop once we get to accuracy key
    break
  else:
    # Add class names and F1 scores to new dictionary
    class_f1_scores[class_names[int(k)]] = v["f1-score"]
class_f1_scores

In [ ]:
# Turn f1 score into dataframes for visualization
import pandas as pd

f1_scores = pd.DataFrame({"class_names": list(class_f1_scores.keys()),
                          "f1-score": list(class_f1_scores.values())}).sort_values("f1-score", ascending=False)
f1_scores

In [ ]:
import matplotlib.pyplot as plt


fix, ax = plt.subplots(figsize=(12,25))
scores = ax.barh(range(len(f1_scores)), f1_scores["f1-score"].values) # get f1-score values
ax.set_yticks(range(len(f1_scores)))
ax.set_yticklabels(f1_scores["class_names"])
ax.set_xlabel("F1 Score")
ax.set_title("F1 Scores for 101 different food classes (predicted by Food Vision mini)")
ax.invert_yaxis() # invert the order of our y axis

## Challenge: Add values to the end of each bar of what the actual f1-score is (hint: use the auto-label function from here)
ax.bar_label(scores, fmt='%.3f')
ax.set_xlim(right=1)  # adjust xlim to fit labels

In [ ]:
pred_classes

## Visualizing predictions on custom images
How does our model go on food images not even in our food data set? (images of our own).

To visualize our model's predictions on our own images, we'll need a function to load and preprocess images (specifically it will need to:
* read in a target image file path using tf.io.read_file()
* Turn the image into a tensor using tf.io.decode_image()
* resize the image tensor to be the same size as the images our model has trained on using tf.image.resize()
* Scale the image to get all of the pixel values between 0 and 1 if necessary

In [ ]:
# Create a function to load and prepare images
def load_and_prep_image(filename, img_shape=(224), scale=True):
  """
  Reads in an image from filename, turns it into a tensor and reshapes it into the specified shape (img_shape, img_shape, color_channels=3)

  Args:
    filename(str): path to target images
    image_shape(int): height/width dimension of target image size
    scale (bool): scale pixel values from 0-255 to 0-1 or not

  Returns:
    Image tensor of shape (image_shape, image_shape, 3)
  """
  # Read in the image
  img = tf.io.read_file(filename)

  # Decode image into tensor
  img = tf.io.decode_image(img, channels=3)

  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])

  # Scale? Yes / No
  if scale:
    # rescale the image (get all values between 0 and 1)
    return (img/255.)
  else:
    return img  # don't need to rescale images for EfficientNet models in TensorFlow

Now we've got a function to load and prepare target images, let's now write some code to visualize images, their target label, and our model's predictions.

Specifically, we'll write some code to:
1. Load a few random images from the test dataset
2. Make predictions on the loaded images
3. Plot the original images along with the model's predictions, prediction probability and truth label


In [ ]:
# Make preds on a series of random images
import os
import random

plt.figure(figsize=(17,10))
for i in range(3):
  # Choose a random image from a random class
  class_name = random.choice(class_names)
  filename = random.choice(os.listdir(test_dir + "/" + class_name))
  filepath = test_dir + class_name + "/" + filename
  print(filepath)
  print(filename)

  # Load the image and make predictions
  img = load_and_prep_image(filepath, scale=False)
  #print(img.shape)
  pred_prob = model.predict(tf.expand_dims(img, axis=0))                # get prediction probabilities array
  pred_class = class_names[pred_prob.argmax()]  # get highest prediction probability index and match it to class names list
  #print(pred_prob)
  #print(pred_class)

  # Plot the images
  plt.subplot(1, 3, i+1)
  plt.imshow(img/255.)
  if class_name==pred_class:   # if predicted class matches truth class, make text green
    title_colour="g"
  else:
    title_colour="r"
  plt.title(f"actual: {class_name}, pred: {pred_class}, prob: {pred_prob.max():.2f}", c=title_colour)
  plt.axis(False)

In [ ]:
import random
random.choice(class_names)

## Finding the most wrong predictions

To find out where our model is most wrong, let's write some code to find out the following:
1. Get all the image file paths in the test dataset using the list_files() method
2. Create a pandas dataframe of the image filepaths, ground truth labels, the predicted classes (from our model), max prediction probabilities.  
3. Use our dataframe to find all the wrong predictions (where the ground truth label doesn't match the prediction).
4. Sort the dataframe based on wrong predictions (have the highest prediction probability predictions at the top).
5. Visualize the images with the highest prediction probabilities but have the wrong prediction.

In [ ]:
# 1. get image files paths
food_classes_101_test_paths = tf.data.Dataset.list_files("/content/101_food_classes_10_percent/test/*/*.jpg", shuffle=False)
food_classes_101_test_paths


In [ ]:
# 2. Create a pandas dataframe of the image filepaths, ground truth labels, the predicted classes (from our model), max prediction probabilities.
import pandas as pd

max_preds_101_dataframe = pd.DataFrame({"filepaths":food_classes_101_test_paths.as_numpy_iterator(),
                                        "y_true":y_labels,
                                        "y_pred":pred_classes,
                                        "pred_conf":pred_probs.max(axis=1), # get the maximum pred probability value
                                        "y_true_classname":[class_names[i] for i in y_labels],
                                        "y_pred_classname":[class_names[i] for i in pred_classes]
                                        })
max_preds_101_dataframe

In [ ]:
max_preds_101_dataframe["pred_correct"] = max_preds_101_dataframe["y_true"] == max_preds_101_dataframe["y_pred"]
max_preds_101_dataframe.head()

In [ ]:
# 4. Sort our dataframe to have most wrong predictions at the top
top_100_wrong = max_preds_101_dataframe[max_preds_101_dataframe["pred_correct"] == False].sort_values("pred_conf", ascending=False)[:100]
top_100_wrong
#max_preds_101_dataframe.filepaths[top_100_wrong.index[:5]]

In [ ]:
# 5. Visualize the data that has the wrong prediction but has the highest pred probability.

plt.figure(figsize=(17,10))
for i in range(10):
  # Choose a random image from a random class
  #class_name = random.choice(class_names)
  #filename = random.choice(os.listdir(test_dir + "/" + class_name))
  #filepath = test_dir + class_name + "/" + filename
  filepath = max_preds_101_dataframe.filepaths[top_100_wrong.index[i]]
  #print(filepath)
  #print(filename)

  # Load the image and make predictions
  img = load_and_prep_image(filepath, scale=False)
  #print(img.shape)
  pred_prob = max_preds_101_dataframe.pred_conf[top_100_wrong.index[i]]                # get prediction probabilities array
  pred_class = max_preds_101_dataframe.y_pred_classname[top_100_wrong.index[i]]  # get highest prediction probability index and match it to class names list
  actual_class = max_preds_101_dataframe.y_true_classname[top_100_wrong.index[i]]
  #print(pred_prob)
  #print(pred_class)

  # Plot the images
  plt.subplot(4, 3, i+1)
  plt.imshow(img/255.)
  plt.tight_layout()
  #if class_name==pred_class:   # if predicted class matches truth class, make text green
  #  title_colour="g"
  #else:
  #  title_colour="r"
  #plt.title(f"actual: {class_name}, pred: {pred_class}, prob: {pred_prob.max():.2f}", c=title_colour)
  plt.title(f"pred conf: {pred_prob}, \n pred class: {pred_class}, actual class: {actual_class}")
  plt.axis(False)

# Test out our big dog model on our own custom datasets

In [ ]:
# Get custom images
#unzip_data("./custom_images_mark.zip")
#custom_food_images = ["./custom_images_mark/" + img_path for img_path in os.listdir("custom_images_mark")]
#custom_food_images

In [ ]:
# Make predictions on and plot custom food images
#for img in custom_food_images:
#  img = load_and_prep_image(img, scale=False) # dont need to scale for our efficientnet
#  pred_prob = model.predict(tf.expand_dims(img, axis=0)) # make prediction on image with shape [1, 224, 224, 4] (same shape as model was trained on)
#  pred_class = class_names[pred_prob.argmax()] # get the index with the highest prediction probability
#  # Plot the appropriate information
#  plt.figure()
#  plt.imshow(img/255.)
#  plt.title(f"pred: {pred_class}, prob: {pred_prob.max():.2f}, filename:")
#  plt.axis(False)

In [ ]:
#!rm -rf ./custom_images_mark/.DS_Store

In [ ]:
# Define the model
model_2 = tf.keras.Model(inputs,outputs)

# Compile the model
new_epochs = 10

model_2.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics="accuracy")

history_all_classes_10_percent_more_epochs = model_2.fit(train_data_all_10_percent,
                                                         batch_size=32,
                                                         epochs=new_epochs,
                                                         validation_data=test_data_all_10_percent,
                                                         validation_steps=int(0.15 * len(test_data_all_10_percent)),
                                                         callbacks=[checkpoint_callback])


In [ ]:
feature_extraction_results_10_epochs = model_2.evaluate(test_data_all_10_percent)
feature_extraction_results_10_epochs

In [ ]:
feature_extraction_results

In [ ]:
all_classes_10_percent_fine_tune_results

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

print("Compute dtype: %s" % policy.compute_dtype)
print("Variable dtype: %s" % policy.variable_dtype)

In [ ]:
# Create data augmentation layer to incorporate it right into our model
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Setup data augmentation
data_augmentation_mixed = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2),
    #preprocessing.Rescale(1/255.), #rescale inputs of images to between 0 and 1, required for models like ResNet50
], name="data_augmentation_mixed")

print(data_augmentation_mixed.dtype_policy)

In [ ]:
# Setup the base model and freeze its layers (this will extract features)
base_model_mixed=tf.keras.applications.EfficientNetB0(include_top=False)
base_model_mixed.trainable = False

#Setup model architecture with trainable top layers
inputs_mixed = layers.Input(shape=(224,224,3), name="input_layer")
x_mixed = data_augmentation(inputs_mixed) # augment images (only happens during training phase)
x_mixed = base_model(x_mixed, training=False) # This will put the base model into inference mode so weights which need to stay frozen stay frozen
x_mixed = layers.GlobalAveragePooling2D(name="global_avg_pool_layer")(x_mixed)
outputs_mixed = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name="output_layer", dtype="float32")(x_mixed)
model_mixed_precision = tf.keras.Model(inputs_mixed, outputs_mixed)
print("Outputs dtype: %s" % outputs_mixed.dtype)

In [ ]:
# Recreate the first model with mixed-precision turned on

# Define the model
model_mixed_precision = tf.keras.Model(inputs,outputs)

# Compile the model
epochs = 5

model_mixed_precision.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics="accuracy",)

history_all_classes_10_percent_mixed_precision = model_mixed_precision.fit(train_data_all_10_percent,
                                                         batch_size=32,
                                                         epochs=epochs,
                                                         validation_data=test_data_all_10_percent,
                                                         validation_steps=int(0.15 * len(test_data_all_10_percent)),
                                                         callbacks=[checkpoint_callback])

In [ ]:
results_mixed_precision = model_mixed_precision.evaluate(test_data_all_10_percent)
results_mixed_precision